# Lab 3: Collecting Data

In [1]:
import numpy as np
import ugradio
from ugradio.timing import nch
import time
from __future__ import division
from astropy import units as u
from astropy.coordinates import SkyCoord

## Cygnus Coordinates

In [2]:
# convert from sexagesimal notation to degree notation

c = SkyCoord('19h59m28.357s', '+40d44m02.10s', frame='icrs')
print(c)

<SkyCoord (ICRS): (ra, dec) in deg
    (299.86815417, 40.73391667)>


In [7]:
def cyg_altaz():
    """Determines altitude and azimuth of Cygnus at current time."""
    ra_deg = 299.86815417
    dec_deg = 40.733916679
    ra_now, dec_now = ugradio.coord.precess(ra_deg, dec_deg)
    cyg_alt, cyg_az = ugradio.coord.get_altaz(ra_now, dec_now)
    return cyg_alt, cyg_az

In [4]:
def cyg_altaz_jd(jd):
    """Determines altitude and azimuth of Cygnus for a given Julian date."""
    ra_deg = 299.86815417
    dec_deg = 40.733916679
    ra_now, dec_now = ugradio.coord.precess(ra_deg, dec_deg, jd)
    cyg_alt, cyg_az = ugradio.coord.get_altaz(ra_now, dec_now, jd)
    return cyg_alt, cyg_az

In [8]:
# for Monday, March 18, 2019

jd0 = 2458560.79167
jd2 = 2458560.87500
jd4 = 2458560.95833
jd6 = 2458561.04167
jd8 = 2458561.12500
jd10 = 2458561.20833
jd12 = 2458561.29167
jd14 = 2458561.37500
jd16 = 2458561.45833
jd18 = 2458561.54167
jd20 = 2458561.62500
jd22 = 2458561.70833
jd24 = 2458561.79167

print 'Altitude, Azimuth of Cygnus at...'
print '00:00 :', cyg_altaz_jd(jd0)
print '02:00 :', cyg_altaz_jd(jd2)
print '04:00 :', cyg_altaz_jd(jd4)
print '06:00 :', cyg_altaz_jd(jd6)
print '08:00 :', cyg_altaz_jd(jd8)
print '10:00 :', cyg_altaz_jd(jd10)
print '12:00 :', cyg_altaz_jd(jd12)
print '14:00 :', cyg_altaz_jd(jd14)
print '16:00 :', cyg_altaz_jd(jd16)
print '18:00 :', cyg_altaz_jd(jd18)
print '20:00 :', cyg_altaz_jd(jd20)
print '22:00 :', cyg_altaz_jd(jd22)
print '22:00 :', cyg_altaz_jd(jd24)

# times in PDT

Altitude, Azimuth of Cygnus at...
00:00 : (-3.9167660071595396, 28.043422048475286)
02:00 : (10.397201609809011, 45.69189885802968)
04:00 : (29.251433117373008, 59.152348379801836)
06:00 : (50.64466437399524, 69.11696157082382)
08:00 : (73.23641220258789, 73.33209025263035)
10:00 : (82.68805161365756, 296.10464457852777)
12:00 : (60.11675353221777, 287.893553917219)
14:00 : (38.07402771817476, 296.21930388044797)
16:00 : (17.95143751898608, 308.1540143067999)
18:00 : (1.4152620525157096, 323.9442837949314)
20:00 : (-9.05737845030826, 344.11900696991125)
22:00 : (-10.902333677157365, 7.062075043151632)
22:00 : (-3.547251247049448, 28.693070247455637)


Cygnus "rises" a little after midnight and "sets" just after 6:00 pm. However, the hills to the North-East (0 < azimuth < 90) may obscure our view for altitudes less than about 30 degrees, and Evans in the North-West (315 < azimuth < 360) may obscure our view for altitudes less than 45 degrees. Our real viewing window is then approximately from 2:00 am to 2:00 pm. Note the measurements for Evans are relative to the west detector.

## The Moon

The Moon rises at 4:46 pm on Monday, March 18 and sets at 6:40 am on the next day. The next full moon is on March 20.

Moon Calendar: https://www.timeanddate.com/moon/usa/berkeley

## Observing with the Interferometer

In [3]:
def diff(a,b):
    return np.abs(a-b)

In [6]:
def sun_coords():
    """Determines altitude and azimuth of the Sun in current time."""
    jd = ugradio.timing.julian_date()
    ra,dec = ugradio.coord.sunpos(jd)
    alt,az = ugradio.coord.get_altaz(ra,dec,jd,nch.lat,nch.lon,nch.alt)
    return (alt,az)

In [8]:
DT = 5
tol = 1

alt, az = sun_alt()

ifm = ugradio.interf.Interferometer()
print("pointing dishes ")
ifm.point(alt,az)
print("first pointing done ")

MaxT = ugradio.timing.julian_date() + 0.00001157*30 # julian_date measured in days, 0.00001157 is one second 


try:

    while True:
        if ugradio.timing.julian_date()>MaxT:
            print("observing successful")
            break
        alt,az = sun_alt()
        ifm.point(alt,az)
        pointing = ifm.get_pointing()
        w_alt,w_az = pointing["ant_w"]
        e_alt,e_az = pointing["ant_e"]
        
        print("target:"+ str(alt)+","+str(az))
        print("west pointed :"+ str(w_alt)+","+str(w_az))
        print("east pointed :"+ str(e_alt)+","+str(e_az))
        if (diff(alt,e_alt)>tol or
            diff(az,e_az)>tol or
            diff(alt,w_alt)>tol or
            diff(az,w_az)>tol ):
            
            raise Exception ("pointing error")
        time.sleep(DT)

except Exception as e:
    print(e)
    
finally:
    ifm.stow()


pointing dishes 
first pointing done 
target:41.6184508983,141.825481686
west pointed :41.6215820312,141.840039062
east pointed :41.572265625,141.814941406
target:41.6382965412,141.87279837
west pointed :41.5776367188,141.840039062
east pointed :41.66015625,141.880859375
target:41.6558132649,141.914628794
west pointed :41.5776367188,141.862011719
east pointed :41.6381835938,141.902832031
target:41.6711762631,141.95136752
west pointed :41.5776367188,141.971875
east pointed :41.66015625,141.924804688
observing successful


In [9]:
#point to the location of the sun
ifm = ugradio.interf.Interferometer()
alt,az = sun_coords()
ifm.point(alt,az)
pointing = ifm.get_pointing()
w_alt,w_az = pointing["ant_w"]
e_alt,e_az = pointing["ant_e"]
print("target:"+ str(alt)+","+str(az))
print("west pointed :"+ str(w_alt)+","+str(w_az))
print("east pointed :"+ str(e_alt)+","+str(e_az))

target:41.7398160506,142.116102617
west pointed :41.9731445312,142.631054688
east pointed :41.5502929688,142.3203125


In [5]:
hpm = ugradio.hp_multi.HP_Multimeter()
hpm.start_recording??

In [17]:
hpm = ugradio.hp_multi.HP_Multimeter()
#hpm.read_voltage()
hpm.start_recording(1, tries = 100)
time.sleep(120) # locally wait for 120 seconds (have to exceed the total processing time)
hpm.end_recording()
#hpm.get_recording_status()
data, times = hpm.get_recording_data()
np.savez("sun-data",volts = data, unitimes = times)

In [16]:
print(data)

[0.000432 0.000435 0.00044  0.000438 0.00044  0.000432 0.000433 0.000429
 0.000431 0.000436 0.000437 0.000436 0.000438 0.000434 0.000437 0.000424
 0.00043  0.000425 0.000439 0.000435 0.000431 0.000433 0.000449 0.000439
 0.000442 0.000442 0.000439 0.000428 0.000436 0.000439 0.000438 0.000422
 0.000436 0.000435 0.000442 0.000432 0.000428 0.000431 0.00042  0.000418
 0.000422 0.000415 0.000425 0.000437 0.000437 0.000431 0.00043  0.000435
 0.000436 0.000428 0.000427 0.000438 0.000433 0.000447 0.00042  0.00043
 0.000438 0.000438 0.000442 0.000442 0.000438 0.000434 0.000432 0.000424
 0.00043  0.000427 0.00043  0.000433 0.000439 0.00045  0.000458 0.000448
 0.000443 0.000445 0.000432 0.000439 0.000443 0.00044  0.000441 0.000438
 0.000446 0.000449 0.000453 0.000445 0.000447 0.000451 0.000447 0.000449
 0.000459 0.000449 0.000426 0.000439 0.000441 0.000434 0.000436 0.000438
 0.000442 0.000438 0.000444 0.000448 0.000451 0.000451 0.000452 0.000444
 0.00044  0.000441 0.000453 0.000455 0.000441 0.0004

In [18]:
ifm.stow()

# always stow detectors when done observing